In [1]:
!curl -sSL https://install.python-poetry.org | python3 -

Retrieving Poetry metadata

The latest version (1.3.2) is already installed.


In [2]:
!poetry install

Installing dependencies from lock file

No dependencies to install or update


In [2]:
!poetry run spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 7.2 MB/s eta 0:00:00m eta 0:00:010:01020m

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
!poetry run python main.py -i 'write a support bundle yaml to get log from pod has api=app, app.kubernetes.io/name=service1， kurl.io/app=hi labels in app namespace'

['write a support bundle yaml to get log from pod has api=app, app.kubernetes.io/name=service1， kurl.io/app=hi labels in app namespace']
<IPython.core.display.Markdown object>


kubectl logs -l api=app,app.kubernetes.io/name=service1,kurl.io/app=hi -n app


========= openai generated yaml: =========

apiVersion: v1
kind: Service
metadata:
  name: service1
  namespace: app
  labels:
    api: app
    app.kubernetes.io/name: service1
    kurl.io/app: hi
spec:
  ports: 
    - port: 80   # Port to expose on the pod's IP address. 
      targetPort: 80 # Port to forward to on the pod targeted by the service. 
      protocol: TCP # Protocol for port. Must be UDP or TCP. Defaults to "TCP".  
  selector: # Selector is a label query over pods that should match the Replicas count.  
    api: app   # Label key to select on.  
    app.kubernetes.io/name : service1 # Label value that must match.  
    kurl.io/app : hi # Label value that must match.




========= support bundle yaml: =========

apiVer

In [6]:
!poetry run python main.py -i '"write a preflight yaml to check status from pod hello in app namespace; fail == [CrashLoopBackOff, Terminating], fail != [Healthy], success == [Ready], warning == [Good]"'

['"write a preflight yaml to check status from pod hello in app namespace', 'fail == [CrashLoopBackOff, Terminating], fail != [Healthy], success == [Ready], warning == [Good]"']
 to check status from pod hello in app namespace


kubectl get pod hello -n app --show-labels --output=wide


========= openai generated yaml: =========

apiVersion: v1
kind: Pod
metadata:
  name: hello-pod
  namespace: app
spec:
  containers:
    - name: hello-container
      image: hello-image
      command: ["/bin/sh", "-c", "while true; do echo 'Checking status...'; sleep 5; done"]


{'fail': ['CrashLoopBackOff', 'Terminating'], 'fail_not_equal': ['Healthy'], 'success': ['Ready'], 'warning': ['Good']}


========= support bundle yaml: =========

apiVersion: troubleshoot.sh/v1beta2
kind: Preflight
metadata:
  name: pods-are-healthy
spec:
  outcomes:
  - fail:
      message: Pod {{ .Namespace }}/{{ .Name }} is in a CrashLoopBackOff state.
      when: == CrashLoopBackOff
  - fail:
      message: Pod {{ .Namespa

In [1]:
!poetry run python main.py -i 'write a support-bundle yaml to collect host memory'

['write a support-bundle yaml to collect host memory']
 usage

kubectl top node --describe-resources=memory


========= support bundle yaml: =========

apiVersion: troubleshoot.sh/v1beta2
kind: SupportBundle
metadata:
  name: get-metrics-from-memory
spec:
  hostCollectors:
  - memory: {}



In [3]:
!poetry run python main.py -i 'write a support bundle yaml to collect host memory; fail: ["< 4g"], warn: ["< 8g"],  pass: [">= 8g"]'

['write a support bundle yaml to collect host memory', 'fail: ["< 4g"], warn: ["< 8g"],  pass: [">= 8g"]']
{'fail': ['< 4g'], 'warn': ['< 8g'], 'pass': ['>= 8g']}


========= support bundle yaml: =========

apiVersion: troubleshoot.sh/v1beta2
kind: SupportBundle
metadata:
  name: get-metrics-from-memory
spec:
  hostAnalyzers:
  - memory:
      checkName: Amount of memory
      outcomes:
      - fail:
          message: If you have less than 4GB of memory, it is recommended that you
            upgrade to 8GB.
          when: < 4g
      - warn:
          message: At least 4GB of memory is required, though 8GB is recommended.
            Currently, less than 8GB is available.
          when: < 8g
      - pass:
          message: At least 4GB of memory is required, with 8GB being the current
            minimum and 8GB being recommended.
          when: '>= 8g'
  hostCollectors:
  - memory: {}



In [3]:
!poetry run python main.py -i 'write a support bundle yaml to collect host cpu; fail: ["count < 2"], warn: ["count < 4"],  pass: ["count >= 4"]'

['write a support bundle yaml to collect host cpu', 'fail: ["count < 2"], warn: ["count < 4"],  pass: ["count >= 4"]']
 usage

kubectl top nodes --describe-containers=true --containers=true --sort-by=cpu
{'fail': ['count < 2'], 'warn': ['count < 4'], 'pass': ['count >= 4']}


========= support bundle yaml: =========

apiVersion: troubleshoot.sh/v1beta2
kind: SupportBundle
metadata:
  name: get-metrics-from-cpu
spec:
  hostAnalyzers:
  - cpu:
      checkName: Amount of cpu
      outcomes:
      - fail:
          message: If the current count of CPU cores is less than 2, at least 2 CPU
            cores are required; however, it is recommended to have 4 CPU cores.
          when: count < 2
      - warn:
          message: It is recommended to have at least 4 CPU cores, although 2 CPU
            cores are the minimum requirement. The current count is less than 4.
          when: count < 4
      - pass:
          message: It is recommended to have at least 4 CPU cores, with the current
  